In [38]:
import math
import collections

import pandas as pd

from keras import regularizers
from keras.models import Model, Sequential
from keras.layers import *

import params
from utils.sequence_data import generator_for_binary_classifier, generator_to_samples_and_targets, generate_samples_from_data
from utils.metrics import print_report_for_binary_classfier
from utils.preprocessing import probs_to_binary_classes
from utils.hyperparams import hyperparams_search
from utils.plot import plot_train_validation_metric

In [39]:
dataset = pd.read_csv("../datasets/data_for_models/dataset_1996-01-01_2019-08-22.csv",
                           header=0, parse_dates=[0], index_col=0)

#params for generator
label_index = len(dataset.columns) -1
input_shape = (params.LOOKBACK//params.STEP, dataset.shape[-1] )

In [40]:
train_max_idx = math.ceil(len(dataset)*params.TRAIN_RATIO)
val_max_idx = math.ceil(len(dataset)*(params.TRAIN_RATIO+params.VAL_RATIO))

# 1 step = 1 batche of samples 
train_steps = (train_max_idx+1 -params.LOOKBACK) // params.BATCH_SIZE
val_steps =  (val_max_idx - train_max_idx - params.LOOKBACK) // params.BATCH_SIZE
test_steps = (len(dataset) - val_max_idx - params.LOOKBACK) // params.BATCH_SIZE

In [41]:
#init generator_for_binary_classifiers
train_gen = generator_for_binary_classifier(dataset.to_numpy(),
                      label_index=label_index,
                      lookback=params.LOOKBACK,
                      delay=params.DELAY,
                      min_index=0,
                      max_index=train_max_idx,
                      shuffle=False,
                      step=params.STEP, 
                      batch_size=params.BATCH_SIZE,
                      interval_label=True)



val_gen = generator_for_binary_classifier(dataset.to_numpy(),
                    label_index=label_index,
                      lookback=params.LOOKBACK,
                      delay=params.DELAY,
                      min_index=train_max_idx+1,
                      max_index=val_max_idx,
                      shuffle=False,
                      step=params.STEP, 
                      batch_size=params.BATCH_SIZE,
                      interval_label=True)

test_gen = generator_for_binary_classifier(dataset.to_numpy(),
                      label_index=label_index,
                      lookback=params.LOOKBACK,
                      delay=params.DELAY,
                      min_index=val_max_idx+1,
                      max_index=None,
                      shuffle=False,
                      step=params.STEP, 
                      batch_size=params.BATCH_SIZE,
                      interval_label=True)


In [42]:
def fc_model(hidden_unit, n_layer, l2_weight, input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    
    for _ in range(n_layer):
        model.add(Dense(hidden_unit, activation='relu', kernel_regularizer=regularizers.l2(l2_weight)))
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
best_params = hyperparams_search(model_func=fc_model, 
                                 input_shape=input_shape,
                                 train_steps=train_steps, 
                                 val_steps=val_steps, 
                                 train_gen=train_gen, 
                                 val_gen=val_gen, 
                                 num_epoch=2000,
                                 hidden_units=[16,32,64], 
                                 n_layers=[2,3], 
                                 l2_weights=[0,0.0005,0.001, 0.0015],
                                 is_verbose=1)
print(best_params)

Epoch 1/2000
63/63 [==============================] - 1s 14ms/step - loss: 0.5797 - acc: 0.7421 - val_loss: 0.3188 - val_acc: 0.9868
Epoch 2/2000
63/63 [==============================] - 1s 10ms/step - loss: 0.5703 - acc: 0.7517 - val_loss: 0.2762 - val_acc: 0.9868
Epoch 3/2000
63/63 [==============================] - 1s 9ms/step - loss: 0.5690 - acc: 0.7470 - val_loss: 0.2710 - val_acc: 0.9868
Epoch 4/2000
63/63 [==============================] - 1s 9ms/step - loss: 0.5641 - acc: 0.7498 - val_loss: 0.2667 - val_acc: 0.9868
Epoch 5/2000
63/63 [==============================] - 1s 10ms/step - loss: 0.5606 - acc: 0.7480 - val_loss: 0.2660 - val_acc: 0.9868
Epoch 6/2000
63/63 [==============================] - 1s 9ms/step - loss: 0.5547 - acc: 0.7488 - val_loss: 0.2732 - val_acc: 0.9868
Epoch 7/2000
63/63 [==============================] - ETA: 0s - loss: 0.5384 - acc: 0.762 - 1s 10ms/step - loss: 0.5441 - acc: 0.7560 - val_loss: 0.2756 - val_acc: 0.9868
Epoch 8/2000
63/63 [==============

63/63 [==============================] - 1s 9ms/step - loss: 0.4691 - acc: 0.7822 - val_loss: 0.2417 - val_acc: 0.9327
Epoch 63/2000
63/63 [==============================] - 1s 9ms/step - loss: 0.4674 - acc: 0.7820 - val_loss: 0.2280 - val_acc: 0.9423
Epoch 64/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.4749 - acc: 0.7775 - val_loss: 0.2212 - val_acc: 0.9435
Epoch 65/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.4755 - acc: 0.7765 - val_loss: 0.2195 - val_acc: 0.9471
Epoch 66/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.4753 - acc: 0.7763 - val_loss: 0.2206 - val_acc: 0.9435
Epoch 67/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.4780 - acc: 0.7736 - val_loss: 0.2240 - val_acc: 0.9423
Epoch 68/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.4720 - acc: 0.7795 - val_loss: 0.2278 - val_acc: 0.9327
Epoch 69/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.4654 -

63/63 [==============================] - 1s 8ms/step - loss: 0.4194 - acc: 0.8120 - val_loss: 0.2680 - val_acc: 0.9123
Epoch 185/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.4160 - acc: 0.8140 - val_loss: 0.2577 - val_acc: 0.9171
Epoch 186/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.4105 - acc: 0.8167 - val_loss: 0.2424 - val_acc: 0.9231
Epoch 187/2000
63/63 [==============================] - 0s 7ms/step - loss: 0.4090 - acc: 0.8189 - val_loss: 0.2381 - val_acc: 0.9255
Epoch 188/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.4196 - acc: 0.8145 - val_loss: 0.2363 - val_acc: 0.9255
Epoch 189/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.4137 - acc: 0.8155 - val_loss: 0.2385 - val_acc: 0.9255
Epoch 190/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.4158 - acc: 0.8157 - val_loss: 0.2379 - val_acc: 0.9255
Epoch 191/2000
63/63 [==============================] - 1s 8ms/step - loss: 0

63/63 [==============================] - 1s 8ms/step - loss: 0.3738 - acc: 0.8331 - val_loss: 0.4338 - val_acc: 0.8317
Epoch 306/2000
63/63 [==============================] - 0s 7ms/step - loss: 0.3767 - acc: 0.8326 - val_loss: 0.3848 - val_acc: 0.8534
Epoch 307/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3728 - acc: 0.8321 - val_loss: 0.4120 - val_acc: 0.8401
Epoch 308/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3749 - acc: 0.8333 - val_loss: 0.4013 - val_acc: 0.8438
Epoch 309/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3752 - acc: 0.8326 - val_loss: 0.3946 - val_acc: 0.8425
Epoch 310/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3728 - acc: 0.8328 - val_loss: 0.3765 - val_acc: 0.8534
Epoch 311/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3688 - acc: 0.8378 - val_loss: 0.3627 - val_acc: 0.8570
Epoch 312/2000
63/63 [==============================] - 0s 8ms/step - loss: 0

Epoch 427/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3716 - acc: 0.8279 - val_loss: 0.2006 - val_acc: 0.9363
Epoch 428/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3726 - acc: 0.8271 - val_loss: 0.2030 - val_acc: 0.9363
Epoch 429/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3798 - acc: 0.8207 - val_loss: 0.2028 - val_acc: 0.9375
Epoch 430/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3833 - acc: 0.8187 - val_loss: 0.2111 - val_acc: 0.9315
Epoch 431/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3861 - acc: 0.8256 - val_loss: 0.2002 - val_acc: 0.9291
Epoch 432/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3929 - acc: 0.8217 - val_loss: 0.2837 - val_acc: 0.7861
Epoch 433/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3855 - acc: 0.8237 - val_loss: 0.2917 - val_acc: 0.7776
Epoch 434/2000
63/63 [==============================] - 0s 8ms

...

63/63 [==============================] - 0s 8ms/step - loss: 0.3373 - acc: 0.8713 - val_loss: 0.3938 - val_acc: 0.8438
Epoch 1217/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3301 - acc: 0.8757 - val_loss: 0.3562 - val_acc: 0.8642
Epoch 1218/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3406 - acc: 0.8686 - val_loss: 0.3710 - val_acc: 0.8678
Epoch 1219/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3271 - acc: 0.8765 - val_loss: 0.3772 - val_acc: 0.8630
Epoch 1220/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3329 - acc: 0.8733 - val_loss: 0.3849 - val_acc: 0.8606
Epoch 1221/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3300 - acc: 0.8733 - val_loss: 0.3832 - val_acc: 0.8594
Epoch 1222/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3332 - acc: 0.8705 - val_loss: 0.3957 - val_acc: 0.8522
Epoch 1223/2000
63/63 [==============================] - 1s 8ms/step - 

Epoch 1277/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3464 - acc: 0.8641 - val_loss: 0.4108 - val_acc: 0.8510
Epoch 1278/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3527 - acc: 0.8584 - val_loss: 0.4351 - val_acc: 0.8389
Epoch 1279/2000
63/63 [==============================] - 0s 7ms/step - loss: 0.3454 - acc: 0.8619 - val_loss: 0.3967 - val_acc: 0.8582
Epoch 1280/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3524 - acc: 0.8581 - val_loss: 0.4241 - val_acc: 0.8413
Epoch 1281/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3408 - acc: 0.8653 - val_loss: 0.3912 - val_acc: 0.8618
Epoch 1282/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3399 - acc: 0.8661 - val_loss: 0.4131 - val_acc: 0.8558
Epoch 1283/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3389 - acc: 0.8663 - val_loss: 0.4101 - val_acc: 0.8582
Epoch 1284/2000
63/63 [==============================] 

Epoch 1398/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3274 - acc: 0.8743 - val_loss: 0.3650 - val_acc: 0.8269
Epoch 1399/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3324 - acc: 0.8713 - val_loss: 0.3803 - val_acc: 0.8185
Epoch 1400/2000
63/63 [==============================] - 0s 7ms/step - loss: 0.3278 - acc: 0.8755 - val_loss: 0.3784 - val_acc: 0.8197
Epoch 1401/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3269 - acc: 0.8765 - val_loss: 0.3588 - val_acc: 0.8281
Epoch 1402/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3314 - acc: 0.8718 - val_loss: 0.3648 - val_acc: 0.8245
Epoch 1403/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3262 - acc: 0.8760 - val_loss: 0.3574 - val_acc: 0.8317
Epoch 1404/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3383 - acc: 0.8663 - val_loss: 0.3552 - val_acc: 0.8329
Epoch 1405/2000
63/63 [==============================] 

Epoch 1519/2000
63/63 [==============================] - 0s 7ms/step - loss: 0.3257 - acc: 0.8760 - val_loss: 0.3644 - val_acc: 0.8173
Epoch 1520/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3181 - acc: 0.8777 - val_loss: 0.3621 - val_acc: 0.8281
Epoch 1521/2000
63/63 [==============================] - 0s 7ms/step - loss: 0.3153 - acc: 0.8790 - val_loss: 0.3707 - val_acc: 0.8221
Epoch 1522/2000
63/63 [==============================] - 1s 10ms/step - loss: 0.3150 - acc: 0.8805 - val_loss: 0.3646 - val_acc: 0.8257
Epoch 1523/2000
63/63 [==============================] - 1s 10ms/step - loss: 0.3154 - acc: 0.8780 - val_loss: 0.3814 - val_acc: 0.8197
Epoch 1524/2000
63/63 [==============================] - 1s 8ms/step - loss: 0.3316 - acc: 0.8686 - val_loss: 0.4161 - val_acc: 0.7969
Epoch 1525/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3466 - acc: 0.8628 - val_loss: 0.3669 - val_acc: 0.8305
Epoch 1526/2000
63/63 [==============================

63/63 [==============================] - 1s 8ms/step - loss: 0.3274 - acc: 0.8715 - val_loss: 0.3692 - val_acc: 0.8305
Epoch 1640/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3209 - acc: 0.8743 - val_loss: 0.3801 - val_acc: 0.8113
Epoch 1641/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3152 - acc: 0.8842 - val_loss: 0.3789 - val_acc: 0.8221
Epoch 1642/2000
63/63 [==============================] - 0s 7ms/step - loss: 0.3117 - acc: 0.8795 - val_loss: 0.3806 - val_acc: 0.8221
Epoch 1643/2000
63/63 [==============================] - 0s 7ms/step - loss: 0.3246 - acc: 0.8750 - val_loss: 0.3954 - val_acc: 0.8173
Epoch 1644/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3275 - acc: 0.8745 - val_loss: 0.3823 - val_acc: 0.8161
Epoch 1645/2000
63/63 [==============================] - 0s 8ms/step - loss: 0.3106 - acc: 0.8817 - val_loss: 0.3965 - val_acc: 0.8149
Epoch 1646/2000
63/63 [==============================] - 1s 9ms/step - 

{'epochs': 1048, 'n_layer': 3, 'hidden_unti': 32, 'l2_weight': 0}

In [10]:
#hyperparameters
num_epoch = best_params['epochs'] 
n_layer= best_params['n_layer'] 
hidden_unit=best_params['hidden_unti']  
l2_weight=best_params['l2_weight']
                    

model = fc_model(hidden_unit, n_layer, l2_weight, input_shape)

# #fit the model
history = model.fit_generator(train_gen,
                    steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=num_epoch, verbose=0)

plot_train_validation_metric(history, 'loss', 'val_loss')

NameError: name 'best_params' is not defined

### We have 2984 positive and 984 negtive cases. 

In [14]:
#Make predictions for train set
    
X, Y = generator_to_samples_and_targets(train_gen, train_steps)    
Y_pred = model.predict(X)
print(collections.Counter(Y))
print_report_for_binary_classfier(Y, probs_to_binary_classes(Y_pred))

Counter({0.0: 3048, 1.0: 984})
F1 score: 0.756196
precision score: 0.873502
recall score: 0.666667
accuracy score: 0.895089
matthews_corrcoef: 0.701127

Confusion matrix:
[[2953   95]
 [ 328  656]]


In [15]:
#Make predictions from dev set
X, Y = generator_to_samples_and_targets(val_gen, val_steps)    
Y_pred = model.predict(X)
print(collections.Counter(Y))
print_report_for_binary_classfier(Y, probs_to_binary_classes(Y_pred))

Counter({0.0: 821, 1.0: 11})
F1 score: 0.033058
precision score: 0.018182
recall score: 0.181818
accuracy score: 0.859375
matthews_corrcoef: 0.016952

Confusion matrix:
[[713 108]
 [  9   2]]


In [16]:
#Make predictions for test set
X, Y = generator_to_samples_and_targets(test_gen, test_steps)    
Y_pred = model.predict(X)
print(collections.Counter(Y))
print_report_for_binary_classfier(Y, probs_to_binary_classes(Y_pred))

Counter({0.0: 802, 1.0: 30})
F1 score: 0.303030
precision score: 0.277778
recall score: 0.333333
accuracy score: 0.944712
matthews_corrcoef: 0.275729

Confusion matrix:
[[776  26]
 [ 20  10]]


# Save the best model 

In [26]:
from keras.models import load_model

model.save(params.BEST_MODEL_PATH)  # creates a HDF5 file 'my_model.h5'